In [1]:
import torch
from torch import nn,optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Loading Data

In [2]:
force = False
training_data = datasets.MNIST('data',train=True,download=force,transform=ToTensor())
test_data = datasets.MNIST('data',train=False,transform=ToTensor())

In [6]:
batch_size = 128
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)
first_data = lambda dataloader : next(iter(dataloader))

X,y = first_data(train_dataloader)

print(f"Shape of X [N, C, H, W]: {X.shape}")
print(f"Shape of y: {y.shape} {y.dtype}")

X,y = first_data(test_dataloader)
print(f"Shape of X [N, C, H, W]: {X.shape}")
print(f"Shape of y: {y.shape} {y.dtype}")

Shape of X [N, C, H, W]: torch.Size([128, 1, 28, 28])
Shape of y: torch.Size([128]) torch.int64
Shape of X [N, C, H, W]: torch.Size([128, 1, 28, 28])
Shape of y: torch.Size([128]) torch.int64


# Device for Training

In [4]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Model Development

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 96, kernel_size=4, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(96, 192, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(192, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avg_pool = nn.AdaptiveAvgPool2d(output_size=(6, 6))
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(128*6*6, 2048),
            nn.ReLU(),
            nn.Linear(2048,1024),
            nn.ReLU(),
            nn.Linear(1024, 10),
            nn.LogSoftmax(),
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.avg_pool(x)
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

# Optimization configuration

In [ ]:
# optimization configurations
loss_fn = nn.CrossEntropyLoss()
epochs = 10
max_learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=max_learning_rate)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer,
                                          max_lr = max_learning_rate,
                                          total_steps = 10 * 60000,)

# Develop Optimization Iteration

In [6]:
def train(dataloader, model, loss_fn, optimizer,scheduler):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        # Print Stats
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Run the iteration

In [8]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer,scheduler)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


C:\Users\Leong Teng Man\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss: 2.302509  [  128/60000]
loss: 0.873454  [12928/60000]
loss: 0.409014  [25728/60000]
loss: 0.157711  [38528/60000]
loss: 0.208298  [51328/60000]
Test Error: 
 Accuracy: 95.3%, Avg loss: 0.149849 

Epoch 2
-------------------------------
loss: 0.062904  [  128/60000]
loss: 0.086543  [12928/60000]
loss: 0.135236  [25728/60000]
loss: 0.095669  [38528/60000]
loss: 0.170879  [51328/60000]
Test Error: 
 Accuracy: 97.3%, Avg loss: 0.093582 

Epoch 3
-------------------------------
loss: 0.027007  [  128/60000]
loss: 0.049372  [12928/60000]
loss: 0.111099  [25728/60000]
loss: 0.086875  [38528/60000]
loss: 0.098707  [51328/60000]
Test Error: 
 Accuracy: 97.6%, Avg loss: 0.085163 

Epoch 4
-------------------------------
loss: 0.021234  [  128/60000]
loss: 0.050863  [12928/60000]
loss: 0.123834  [25728/60000]
loss: 0.065911  [38528/60000]
loss: 0.078001  [51328/60000]
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.048823 

Epoch 5
-------------------------------
loss: 0.006444  [  128/60000]
lo

# Save and Load Model

In [9]:
torch.save(model.state_dict(), "model/mnist_cnn.pth")
print("Saved PyTorch Model State to model/mnist_cnn.pth")

Saved PyTorch Model State to model/mnist_cnn.pth


In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model/mnist_cnn.pth"))
model.to(device);

In [11]:
test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 98.3%, Avg loss: 0.082364 

